In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import json
import re
import ast
import itertools

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

In [3]:
principal_cells = pd.read_pickle('../data/rodent_3d_dendrites_br-ct-filter-1_principal.pkl')
interneurons = pd.read_pickle('../data/rodent_3d_dendrites_br-ct-filter-1_interneuron.pkl')
print(len(principal_cells), len(interneurons))

17894 7294


# things to consider
### principal vs. interneuron
### brain region (cortical vs. subcortical
### cell type
### layer

In [4]:
principal_cells.br1.value_counts()

neocortex              10480
hippocampus             3211
basal ganglia           1471
retina                  1335
main olfactory bulb      491
amygdala                 356
ventral striatum         295
spinal cord              109
brainstem                106
cerebellum                40
Name: br1, dtype: int64

In [5]:
pd.crosstab(principal_cells.rough_layer, principal_cells.br1) # no layer info in other brain regions

br1,neocortex
rough_layer,
layer 1,43
layer 2-3,2585
layer 4,342
layer 5-6,2176


In [6]:
interneurons.br1.value_counts()

neocortex              5656
retina                  816
hippocampus             433
spinal cord             211
amygdala                 62
main olfactory bulb      45
cerebellum               31
basal ganglia            28
brainstem                12
Name: br1, dtype: int64

# Focus on principal cells first

# 01 - Cortex

In [7]:
principal_cortex = principal_cells[principal_cells['br1'] == 'neocortex']

In [8]:
principal_cortex.br2.value_counts(dropna=False)

occipital             4436
somatosensory         2073
medial prefrontal     1843
frontal                930
anterior cingulate     475
prefrontal             346
NaN                    193
Multiple                75
perirhinal              43
temporal                41
dorsomedial             22
primary auditory         2
presubiculum             1
Name: br2, dtype: int64

## Filter br2

In [9]:
# dorsalmedial cells are bad reconstructions. 
# perirhinal cells: half of them are bad recon. 
# temporal lobe CR cells have simple dendrites but complicated axons
# multiple are bad reconstructions
# ALso delete the last two

In [10]:
br2_not_filter = ['Multiple', 'temporal', 'dorsomedial', 'primary auditory', 'presubiculum', 'perirhinal', None]
br2_not_filter

['Multiple',
 'temporal',
 'dorsomedial',
 'primary auditory',
 'presubiculum',
 'perirhinal',
 None]

In [11]:
principal_cortex = principal_cortex[~principal_cortex.br2.isin(br2_not_filter)]

In [12]:

principal_cortex.loc[:,'br2'][principal_cortex.br2=='prefrontal'] = \
    principal_cortex.loc[:,'br2'][principal_cortex.br2=='prefrontal'].map({'prefrontal': 'medial prefrontal'})

In [13]:
principal_cortex.br2.value_counts(dropna=False)

occipital             4436
medial prefrontal     2189
somatosensory         2073
frontal                930
anterior cingulate     475
Name: br2, dtype: int64

## remove layer 1

In [14]:
principal_cortex.exact_layer.value_counts(dropna=False)

NaN          5029
layer 5      1604
layer 2-3    1431
layer 3      1062
layer 6       503
layer 4       340
layer 2        87
layer 5-6      47
Name: exact_layer, dtype: int64

In [15]:
principal_cortex = principal_cortex[principal_cortex.exact_layer != 'layer 1']
principal_cortex.exact_layer.value_counts(dropna=False)

NaN          5029
layer 5      1604
layer 2-3    1431
layer 3      1062
layer 6       503
layer 4       340
layer 2        87
layer 5-6      47
Name: exact_layer, dtype: int64

## filter principal cell subtypes

In [16]:
principal_cortex.cell_type_2_curated.value_counts(dropna=False)

pyramidal    9683
None          291
NaN            89
stellate       40
Name: cell_type_2_curated, dtype: int64

In [17]:
pd.crosstab(principal_cortex.cell_type_2_curated, principal_cortex.br3)

br3,Subplate,anteromedial visual,barrel,infralimbic,lateral visual,medial,orbital gyri,posteromedial visual,precentral gyrus,prelimbic,primary motor,primary somatosensory,primary visual,rostrolateral visual,secondary motor,secondary somatosensory
cell_type_2_curated,,,,,,,,,,,,,,,,
None,291,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
pyramidal,3,1,439,36,2,126,30,2,16,706,29,328,2544,1,143,4
stellate,0,0,29,0,0,0,0,0,0,0,0,0,0,0,0,0


In [18]:
# most of the None's are subplate neurons. exclude them. 

In [19]:
principal_cortex = principal_cortex[~principal_cortex.cell_type_2_curated.isin(['glutamatergic', 'projection', 'None'])]
principal_cortex = principal_cortex[~pd.isnull(principal_cortex.cell_type_2_curated)]
principal_cortex = principal_cortex[principal_cortex.br3 != 'Subplate']
principal_cortex.cell_type_2_curated.value_counts(dropna=False)

pyramidal    9680
stellate       40
Name: cell_type_2_curated, dtype: int64

In [20]:
principal_cortex.to_pickle('../data/rodent_3d_dendrites_br-ct-filter-2_principal_cortex.pkl')

In [21]:
pd.crosstab(principal_cortex.br2, principal_cortex.br3)

br3,anteromedial visual,barrel,infralimbic,lateral visual,medial,orbital gyri,posteromedial visual,precentral gyrus,prelimbic,primary motor,primary somatosensory,primary visual,rostrolateral visual,secondary motor,secondary somatosensory
br2,,,,,,,,,,,,,,,
frontal,0,0,0,0,0,30,0,16,0,29,0,0,0,143,0
medial prefrontal,0,0,36,0,126,0,0,0,706,0,0,0,0,0,0
occipital,1,0,0,2,0,0,2,0,0,0,0,2544,1,0,0
somatosensory,0,468,0,0,0,0,0,0,0,0,328,0,0,0,4


In [22]:
pd.crosstab(principal_cortex.br2, principal_cortex.rough_layer)

rough_layer,layer 2-3,layer 4,layer 5-6
br2,,,
anterior cingulate,475,0,0
frontal,618,0,312
medial prefrontal,824,0,891
occipital,209,55,157
somatosensory,454,285,728


In [23]:
pd.crosstab(principal_cortex.br2, principal_cortex.cell_type_2_curated)

cell_type_2_curated,pyramidal,stellate
br2,,
anterior cingulate,475,0
frontal,930,0
medial prefrontal,2189,0
occipital,4436,0
somatosensory,1650,40


In [24]:
pd.crosstab(principal_cortex.br2, principal_cortex.ds_layer, margins=True, dropna=False)

ds_layer,deep,superficial,All
br2,,,
anterior cingulate,0,475,475
frontal,312,618,930
medial prefrontal,891,824,2189
occipital,1290,1320,4436
somatosensory,1013,454,1690
All,3506,3691,9720


In [25]:
principal_cortex.ds_layer.value_counts(dropna=False)

superficial    3691
deep           3506
NaN            2523
Name: ds_layer, dtype: int64

In [26]:
pd.crosstab(principal_cortex.cell_type_2_curated,principal_cortex.br2)

br2,anterior cingulate,frontal,medial prefrontal,occipital,somatosensory
cell_type_2_curated,,,,,
pyramidal,475,930,2189,4436,1650
stellate,0,0,0,0,40


In [27]:
principal_cortex.cell_type_2_curated.value_counts(dropna=False)

pyramidal    9680
stellate       40
Name: cell_type_2_curated, dtype: int64

# 02 - hippocampus

In [28]:
principal_hippo = principal_cells[principal_cells['br1'] == 'hippocampus']

In [29]:
principal_hippo.br2.value_counts(dropna=False)

dentate gyrus    1911
CA1               875
CA3               188
NaN               118
subiculum         111
CA2                 8
Name: br2, dtype: int64

# filter br2 and cell type

In [30]:
principal_hippo = principal_hippo[~principal_hippo.br2.isin(['CA2', None])]
principal_hippo.br2.value_counts(dropna=False)

dentate gyrus    1911
CA1               875
CA3               188
subiculum         111
Name: br2, dtype: int64

In [31]:
principal_hippo.cell_type_2_curated.value_counts(dropna=False)

granule          1881
pyramidal        1169
Cajal-Retzius      29
NaN                 6
Name: cell_type_2_curated, dtype: int64

In [32]:
principal_hippo = principal_hippo[~principal_hippo.cell_type_2_curated.isin(['Cajal-Retzius', None])]
principal_hippo = principal_hippo[~pd.isnull(principal_hippo.cell_type_2_curated)]
principal_hippo.cell_type_2_curated.value_counts(dropna=False)

granule      1881
pyramidal    1169
Name: cell_type_2_curated, dtype: int64

In [33]:
pd.crosstab(principal_hippo.br2, principal_hippo.cell_type_2_curated, dropna=False, margins=True)

cell_type_2_curated,granule,pyramidal,All
br2,,,
CA1,0,870,870
CA3,0,188,188
dentate gyrus,1881,0,1881
subiculum,0,111,111
All,1881,1169,3050


## can use `DG granule` class and `hippocampus + pyramidal` class

# 03 - basal ganglia 
## -- include br1==ventral striatum into basal ganglia

In [34]:
principal_vs = principal_cells[principal_cells['br1'] == 'ventral striatum']
principal_vs.br2.value_counts(dropna=False)

nucleus accumbens    295
Name: br2, dtype: int64

In [35]:
principal_vs.br3.value_counts(dropna=False)

shell    293
Core       1
NaN        1
Name: br3, dtype: int64

In [36]:
principal_vs.br4.value_counts(dropna=False)

NaN    295
Name: br4, dtype: int64

In [37]:
principal_cells.loc[principal_cells['br1'] == 'ventral striatum','br4'] = principal_vs.br3
principal_cells.loc[principal_cells['br1'] == 'ventral striatum','br3'] = principal_vs.br2
principal_cells.loc[principal_cells['br1'] == 'ventral striatum','br2'] = principal_vs.br1
principal_cells.loc[principal_cells['br1'] == 'ventral striatum','br1'] = 'basal ganglia'

In [38]:
principal_cells[principal_cells['br1'] == 'ventral striatum'] # now we don't have 'ventral striatum' as br1 anymore

,age_classification,archive,attributes,brain_region,cell_type,deposition_date,domain,experiment_condition,magnification,max_age,max_weight,min_age,min_weight,neuron_id,note,objective_type,original_format,physical_Integrity,protocol,reconstruction_software,reference_pmid,scientific_name,shrinkage_corrected,shrinkage_reported,slicing_direction,slicing_thickness,soma_surface,species,stain,strain,upload_date,3D,angles,bif_ampl_local,branch_Order,contraction,depth,diameter,eucDistance,fractal_Dim,fragmentation,height,length,n_bifs,n_branch,n_stems,neuron_name,partition_asymmetry,pathDistance,pk_classic,soma_Surface,surface,volume,width,br1,br2,br3,br4,ds_layer,hemisphere,raw_layer,exact_layer,rough_layer,cell_type_1,cell_type_2,cell_type_2_curated


# basal ganglia

In [39]:
principal_bg = principal_cells[principal_cells['br1'] == 'basal ganglia']
principal_bg.br2.value_counts(dropna=False)

ventral striatum     1100
striatum              392
nucleus accumbens     163
globus pallidus        89
substantia nigra       22
Name: br2, dtype: int64

## put nucleus accumbens under ventral stiatum

In [40]:
principal_bg_accumbens = principal_bg[principal_cells['br2'] == 'nucleus accumbens']
principal_cells.loc[(principal_cells['br1'] == 'basal ganglia') & (principal_cells['br2'] == 'nucleus accumbens'),'br4'] = \
    principal_bg_accumbens.br3
principal_cells.loc[(principal_cells['br1'] == 'basal ganglia') & (principal_cells['br2'] == 'nucleus accumbens'),'br3'] = \
    principal_bg_accumbens.br2
principal_cells.loc[(principal_cells['br1'] == 'basal ganglia') & (principal_cells['br2'] == 'nucleus accumbens'),'br2'] = \
    'ventral striatum'

/home/yueqi/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


In [41]:
principal_bg = principal_cells[principal_cells['br1'] == 'basal ganglia']
principal_bg.br2.value_counts(dropna=False)

ventral striatum    1263
striatum             392
globus pallidus       89
substantia nigra      22
Name: br2, dtype: int64

## change 'striatum' to 'dorsal striatum' since they're either dorsolateral or dorsomedial

In [42]:
principal_cells.loc[(principal_cells['br1'] == 'basal ganglia') & (principal_cells['br2'] == 'striatum'),'br2'] = \
    'dorsal striatum'

In [43]:
principal_bg = principal_cells[principal_cells['br1'] == 'basal ganglia']
principal_bg.br2.value_counts(dropna=False)

ventral striatum    1263
dorsal striatum      392
globus pallidus       89
substantia nigra      22
Name: br2, dtype: int64

In [44]:
pd.crosstab(principal_bg.br2, principal_bg.br3, margins=True)

br3,Contralateral,Ipsilateral,bed nucleus of the stria terminalis,dorsolateral,dorsomedial,nucleus accumbens,pars compacta,All
br2,,,,,,,,
dorsal striatum,12,13,0,6,194,0,0,225
globus pallidus,0,0,83,0,0,0,0,83
substantia nigra,0,0,0,0,0,0,6,6
ventral striatum,0,0,0,0,0,1263,0,1263
All,12,13,83,6,194,1263,6,1577


In [45]:
pd.crosstab(principal_bg.br3, principal_bg.br4.astype(str), margins=True)

br4,Core,None,['dorsolateral'],['dorsomedial'],"['medial', 'shell']",['shell'],shell,All
br3,,,,,,,,
Contralateral,0,0,7,5,0,0,0,12
Ipsilateral,0,0,5,8,0,0,0,13
bed nucleus of the stria terminalis,0,83,0,0,0,0,0,83
dorsolateral,0,6,0,0,0,0,0,6
dorsomedial,0,194,0,0,0,0,0,194
nucleus accumbens,90,441,0,0,36,329,367,1263
pars compacta,0,6,0,0,0,0,0,6
All,90,730,12,13,36,329,367,1577


## most of ventral striatum/nucleus accumbens cells are medium spiny neurons
## most of dorsal striatum cells are medium spiny neurons

In [46]:
principal_bg = principal_bg[principal_bg.cell_type_2_curated != 'None']
pd.crosstab(principal_bg.br2, principal_bg.cell_type_2_curated, margins=True)

cell_type_2_curated,Aspiny,Neuroblast,dopaminergic,medium spiny,All
br2,,,,,
dorsal striatum,0,25,0,367,392
globus pallidus,0,0,0,83,83
substantia nigra,0,0,22,0,22
ventral striatum,146,0,0,1117,1263
All,146,25,22,1567,1760


## can use `basal ganglia` + `medium spiny` as a class
### may further subclass into dorsal vs. ventral

In [47]:
principal_cells[(principal_cells['br2'] == 'globus pallidus') & (principal_cells.cell_type_2_curated == 'medium spiny')]

,age_classification,archive,attributes,brain_region,cell_type,deposition_date,domain,experiment_condition,magnification,max_age,max_weight,min_age,min_weight,neuron_id,note,objective_type,original_format,physical_Integrity,protocol,reconstruction_software,reference_pmid,scientific_name,shrinkage_corrected,shrinkage_reported,slicing_direction,slicing_thickness,soma_surface,species,stain,strain,upload_date,3D,angles,bif_ampl_local,branch_Order,contraction,depth,diameter,eucDistance,fractal_Dim,fragmentation,height,length,n_bifs,n_branch,n_stems,neuron_name,partition_asymmetry,pathDistance,pk_classic,soma_Surface,surface,volume,width,br1,br2,br3,br4,ds_layer,hemisphere,raw_layer,exact_layer,rough_layer,cell_type_1,cell_type_2,cell_type_2_curated
25609,young adult,Smith,"Diameter, 3D, Angles","[basal ganglia, globus pallidus, bed nucleus o...","[principal cell, GABAergic, medium spiny]",2016-01-02,"Dendrites, Soma, No Axon",Not reported,60,60.0,None,60.0,None,68178,,Not reported,Neurolucida.dat,Dendrites Moderate,in vitro,Neurolucida,[25623037],rattus norvegicus,,N,Not reported,200.00,6855.39,rat,Golgi-Cox,Sprague-Dawley,2017-07-19,True,True,65.8280,1.0,0.961971,11.37,1.037300,127.1660,1.00597,244.0,113.496,414.734,2.0,7.0,3.0,BST2A1-1,0.000000,132.3990,2.000000,6855.39,1342.21,352.552,163.2170,basal ganglia,globus pallidus,bed nucleus of the stria terminalis,None,None,None,None,NaN,NaN,principal cell,"[GABAergic, medium spiny]",medium spiny
25610,young adult,Smith,"Diameter, 3D, Angles","[basal ganglia, globus pallidus, bed nucleus o...","[principal cell, GABAergic, medium spiny]",2016-01-02,"Dendrites, Soma, No Axon",Not reported,60,60.0,None,60.0,None,68179,,Not reported,Neurolucida.dat,Dendrites Moderate,in vitro,Neurolucida,[25623037],rattus norvegicus,,N,Not reported,200.00,7763.3,rat,Golgi-Cox,Sprague-Dawley,2017-07-19,True,True,48.4592,2.0,0.967656,26.35,0.947028,154.2360,1.00816,471.0,251.381,593.142,4.0,11.0,3.0,BST2A1-2,0.500000,165.9350,1.845640,7763.30,1801.55,439.366,157.4110,basal ganglia,globus pallidus,bed nucleus of the stria terminalis,None,None,None,None,NaN,NaN,principal cell,"[GABAergic, medium spiny]",medium spiny
25611,young adult,Smith,"Diameter, 3D, Angles","[basal ganglia, globus pallidus, bed nucleus o...","[principal cell, GABAergic, medium spiny]",2016-01-02,"Dendrites, Soma, No Axon",Not reported,60,60.0,None,60.0,None,68180,,Not reported,Neurolucida.dat,Dendrites Moderate,in vitro,Neurolucida,[25623037],rattus norvegicus,,N,Not reported,200.00,3258.37,rat,Golgi-Cox,Sprague-Dawley,2017-07-19,True,True,64.0752,1.0,0.953911,19.03,1.214530,83.9752,1.01461,214.0,126.500,314.318,2.0,8.0,4.0,BST2A1-3,0.000000,87.9325,1.637720,3258.37,1208.37,389.296,58.6471,basal ganglia,globus pallidus,bed nucleus of the stria terminalis,None,None,None,None,NaN,NaN,principal cell,"[GABAergic, medium spiny]",medium spiny
25612,young adult,Smith,"Diameter, 3D, Angles","[basal ganglia, globus pallidus, bed nucleus o...","[principal cell, GABAergic, medium spiny]",2016-01-02,"Dendrites, Soma, No Axon",Not reported,60,60.0,None,60.0,None,68181,,Not reported,Neurolucida.dat,Dendrites Moderate,in vitro,Neurolucida,[25623037],rattus norvegicus,,N,Not reported,200.00,12735.9,rat,Golgi-Cox,Sprague-Dawley,2017-07-19,True,True,42.7275,2.0,0.965751,7.59,1.061570,144.4940,1.00912,293.0,147.490,441.177,2.0,7.0,3.0,BST2A1-4,0.500000,151.0860,1.689060,12735.90,1554.67,463.756,178.5500,basal ganglia,globus pallidus,bed nucleus of the stria terminalis,None,None,None,None,NaN,NaN,principal cell,"[GABAergic, medium spiny]",medium spiny
25613,young adult,Smith,"Diameter, 3D, Angles","[basal ganglia, globus pallidus, bed nucleus o...","[principal cell, GABAergic, medium spiny]",2016-01-02,"Dendrites, Soma, Axon",Not reported,60,60.0,None,60.0,None,68182,,Not reported,Neurolucida.dat,"Dendrites Moderate, Axon Incomplete",in vitro,Neurolucida,[25623037],rattus norvegicus,,N,Not reported,200.00,11663.8,rat,Golgi-Cox,Sprague-Dawley,2017-07-19,True,

# 04 - retina

In [48]:
principal_retina = principal_cells[principal_cells['br1'] == 'retina']
principal_retina.br2.value_counts(dropna=False)

ganglion layer           859
NaN                      420
inner plexiform layer     56
Name: br2, dtype: int64

In [49]:
principal_retina.br3.value_counts(dropna=False)

NaN            1286
sublamina a      29
sublamina b      20
Name: br3, dtype: int64

In [50]:
principal_retina.cell_type_2_curated.value_counts(dropna=False)

ganglion    1333
NaN            2
Name: cell_type_2_curated, dtype: int64

In [51]:
pd.crosstab(principal_retina.br2, principal_retina.cell_type_2_curated, dropna=False)

cell_type_2_curated,ganglion
br2,
ganglion layer,859
inner plexiform layer,56


In [52]:
principal_retina = principal_retina[~pd.isnull(principal_retina.cell_type_2_curated)]
principal_retina.cell_type_2_curated.value_counts(dropna=False)

ganglion    1333
Name: cell_type_2_curated, dtype: int64

In [54]:
principal_retina = principal_retina[principal_retina.domain!='Neurites, No Soma']

In [55]:
pd.crosstab(principal_retina.br2, principal_retina.cell_type_2.astype(str), dropna=False)

cell_type_2,"['A1 type', 'ON type', 'ganglion']","['A2 inner type', 'ON type', 'ganglion']","['A2 outer type', 'OFF type', 'ganglion']","['B1 type', 'OFF type', 'ganglion']","['B3 outer type', 'OFF type', 'ganglion']","['C2 inner type', 'ON type', 'ganglion']","['C2 outer type', 'OFF type', 'ganglion']","['Monostratified', 'ganglion']","['OFF type', 'ganglion']","['ON type', 'ganglion']","['bistratified', 'ganglion']","['cluster A', 'ganglion']","['cluster B', 'ganglion']","['cluster C', 'ganglion']","['cluster D', 'ganglion']","['cluster E', 'ganglion']","['cluster F', 'ganglion']","['cluster G', 'ganglion']","['cluster H', 'ganglion']","['cluster I', 'ganglion']","['cluster U', 'ganglion']","['cluster V', 'ganglion']","['cluster W', 'ganglion']","['cluster X', 'ganglion']","['cluster Y', 'ganglion']","['cluster Z', 'ganglion']","['ganglion', 'OFF type']","['ganglion', 'ON type', 'orientation selective']","['ganglion', 'ON type']","['ganglion', 'On-Off direction selective ganglion cell']","['ganglion', 'W3b']",['ganglion']
br2,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ganglion layer,0,0,0,0,0,0,0,133,1,1,48,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,676
inner plexiform layer,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,29,0,20,0,0,0


## can use `retina` + `ganglion cell` as a class

In [56]:
principal_cells[(principal_cells['br1'] == 'retina') & (principal_cells.cell_type_2_curated == 'ganglion')]

,age_classification,archive,attributes,brain_region,cell_type,deposition_date,domain,experiment_condition,magnification,max_age,max_weight,min_age,min_weight,neuron_id,note,objective_type,original_format,physical_Integrity,protocol,reconstruction_software,reference_pmid,scientific_name,shrinkage_corrected,shrinkage_reported,slicing_direction,slicing_thickness,soma_surface,species,stain,strain,upload_date,3D,angles,bif_ampl_local,branch_Order,contraction,depth,diameter,eucDistance,fractal_Dim,fragmentation,height,length,n_bifs,n_branch,n_stems,neuron_name,partition_asymmetry,pathDistance,pk_classic,soma_Surface,surface,volume,width,br1,br2,br3,br4,ds_layer,hemisphere,raw_layer,exact_layer,rough_layer,cell_type_1,cell_type_2,cell_type_2_curated
1916,adult,Chalupa,"Diameter, 3D, Angles","[retina, ganglion layer]","[Monostratified, ganglion, principal cell]",2009-06-29,"Dendrites, Soma, Axon",Control,None,None,None,None,None,5312,dendrites in layers 1 and 5,Not reported,Neurolucida.asc,"Dendrites Complete, Axon Incomplete",in vitro,Neurolucida,[16626866],,,Not reported,whole mount,Not applicable,359.31,mouse,lucifer yellow,C57BL/6,2009-09-04,True,True,82.3600,5.0,0.880000,11.89,0.640000,326.7500,1.02822,3935.0,416.8500,5240.910,32.0,71.0,7.0,cell-2-trace,0.350000,348.720,1.75000,359.3100,11147.50,2295.380,285.7100,retina,ganglion layer,None,None,None,None,None,NaN,NaN,principal cell,"[Monostratified, ganglion]",ganglion
1917,adult,Chalupa,"Diameter, 3D, Angles","[retina, ganglion layer]","[Monostratified, ganglion, principal cell]",2009-06-29,"Dendrites, Soma, Axon",Control,None,None,None,None,None,5313,"dendrites in layers 1,2,4 and 5",Not reported,Neurolucida.asc,"Dendrites Complete, Axon Incomplete",in vitro,Neurolucida,[16626866],,,Not reported,whole mount,Not applicable,101.57,mouse,lucifer yellow,C57BL/6,2009-09-04,True,True,85.7300,7.0,0.890000,24.49,0.350000,188.8100,1.02000,1684.0,268.1900,4962.180,47.0,98.0,4.0,cell-6-trace,0.350000,261.380,2.00000,101.5700,5456.21,477.410,275.3200,retina,ganglion layer,None,None,None,None,None,NaN,NaN,principal cell,"[Monostratified, ganglion]",ganglion
1918,adult,Chalupa,"Diameter, 3D, Angles","[retina, ganglion layer]","[Monostratified, ganglion, principal cell]",2009-06-29,"Dendrites, Soma, Axon",Control,None,None,None,None,None,5314,dendrites in layers 1 and 4,Not reported,Neurolucida.asc,"Dendrites Complete, Axon Incomplete",in vitro,Neurolucida,[16626866],,,Not reported,whole mount,Not applicable,564.2,mouse,lucifer yellow,C57BL/6,2009-09-04,True,True,86.3000,7.0,0.840000,28.35,0.520000,198.9300,1.03000,2060.0,287.0100,3513.840,31.0,65.0,3.0,cell-7-trace,0.370000,293.720,1.90000,564.2000,5766.56,800.560,266.2700,retina,ganglion layer,None,None,None,None,None,NaN,NaN,principal cell,"[Monostratified, ganglion]",ganglion
1919,adult,Chalupa,"Diameter, 3D, Angles","[retina, ganglion layer]","[Monostratified, ganglion, principal cell]",2009-06-29,"Dendrites, Soma, Axon",Control,None,None,None,None,None,5315,dendrites in layers 1 and 4,Not reported,Neurolucida.asc,"Dendrites Complete, Axon Incomplete",in vitro,Neurolucida,[16626866],,,Not reported,whole mount,Not applicable,708.9,mouse,lucifer yellow,C57BL/6,2009-09-04,True,True,79.6100,10.0,0.790000,25.18,0.730000,323.6200,1.05000,3049.0,301.0700,4598.180,35.0,72.0,2.0,cell-8-trace,0.580000,463.100,1.93000,708.9000,11478.40,3153.660,281.6000,retina,ganglion layer,None,None,None,None,None,NaN,NaN,principal cell,"[Monostratified, ganglion]",ganglion
1920,adult,Chalupa,"Diameter, 3D, Angles","[retina, ganglion layer]","[Monostratified, ganglion, principal cell]",2009-06-29,"Dendrites, Soma, Axon",Control,None,None,None,None,None,5316,dendrites in layers 1 and 5,Not reported,Neurolucida.asc,"Dendrites Complete, Axon Incomplete",in vitro,Neurolucida,[16626866],,,Not reported,whole mount,Not applicable,74.13,mouse,lucifer yellow,C57BL/6,2009-09-04,True,True,91.3200,6.0,0.850000,16.89,0.660000,203.3700,1.03000,1960.0,276.9100,4057.460,24.0,56.

# 05 - main olfactory bulb

In [57]:
principal_ob = principal_cells[principal_cells['br1'] == 'main olfactory bulb']
principal_ob.br2.value_counts(dropna=False)

NaN                         342
stratum granulosum          106
mitral layer                 20
glomerular layer             15
external plexiform layer      8
Name: br2, dtype: int64

In [58]:
principal_ob = principal_ob[~pd.isnull(principal_ob.cell_type_2_curated)]
principal_ob.cell_type_2_curated.value_counts(dropna=False)

granule    282
mitral      52
tufted      33
Name: cell_type_2_curated, dtype: int64

In [59]:
pd.crosstab(principal_ob.br2, principal_ob.cell_type_2_curated, dropna=False, margins=True)

cell_type_2_curated,granule,mitral,tufted,All
br2,,,,
external plexiform layer,0,0,8,8
mitral layer,0,20,0,20
All,282,52,33,367


# can use ob + granule as a class, not many cells though

In [60]:
principal_ob[principal_ob.cell_type_2_curated=='granule']

,age_classification,archive,attributes,brain_region,cell_type,deposition_date,domain,experiment_condition,magnification,max_age,max_weight,min_age,min_weight,neuron_id,note,objective_type,original_format,physical_Integrity,protocol,reconstruction_software,reference_pmid,scientific_name,shrinkage_corrected,shrinkage_reported,slicing_direction,slicing_thickness,soma_surface,species,stain,strain,upload_date,3D,angles,bif_ampl_local,branch_Order,contraction,depth,diameter,eucDistance,fractal_Dim,fragmentation,height,length,n_bifs,n_branch,n_stems,neuron_name,partition_asymmetry,pathDistance,pk_classic,soma_Surface,surface,volume,width,br1,br2,br3,br4,ds_layer,hemisphere,raw_layer,exact_layer,rough_layer,cell_type_1,cell_type_2,cell_type_2_curated
17957,young adult,Guthrie,"No Diameter, 3D, Angles",[main olfactory bulb],"[principal cell, granule]",2016-01-18,"Dendrites, Soma, No Axon",Transgenic,100,3.0,None,2.0,None,52444,,oil,Neurolucida.dat,Dendrites Complete,in vitro,Neurolucida,[26211445],None,N,Not reported,coronal,160.00,11289.2,mouse,Golgi-Cox,C57BL/6-Tg-(CAM2a-BDNF)A9stl/J,2017-03-30,True,True,78.8229,3.0,0.907709,12.79,0.07,86.0600,1.02945,253.0,77.4798,191.106,4.0,9.0,1.0,R4Grid1Sec3Cell15,0.333333,107.616,2.0,11289.2000,42.0264,0.735462,45.33030,main olfactory bulb,None,None,None,None,None,None,NaN,NaN,principal cell,[granule],granule
17958,young adult,Guthrie,"No Diameter, 3D, Angles",[main olfactory bulb],"[principal cell, granule]",2016-01-18,"Dendrites, Soma, No Axon",Transgenic,100,3.0,None,2.0,None,52445,,oil,Neurolucida.dat,Dendrites Complete,in vitro,Neurolucida,[26211445],None,N,Not reported,coronal,160.00,3571.79,mouse,Golgi-Cox,C57BL/6-Tg-(CAM2a-BDNF)A9stl/J,2017-03-30,True,True,66.0289,2.0,0.864302,22.69,0.07,104.0330,1.03254,234.0,94.3588,247.302,2.0,6.0,2.0,R4Grid4sec2Cell21,0.500000,113.274,2.0,3571.7900,54.3846,0.951731,36.73870,main olfactory bulb,None,None,None,None,None,None,NaN,NaN,principal cell,[granule],granule
17959,young adult,Guthrie,"No Diameter, 3D, Angles",[main olfactory bulb],"[principal cell, granule]",2016-01-18,"Dendrites, Soma, No Axon",Transgenic,100,3.0,None,2.0,None,52446,,oil,Neurolucida.dat,Dendrites Complete,in vitro,Neurolucida,[26211445],None,N,Not reported,coronal,160.00,2936.2,mouse,Golgi-Cox,C57BL/6-Tg-(CAM2a-BDNF)A9stl/J,2017-03-30,True,True,65.6071,1.0,0.935166,14.02,0.07,127.7580,1.01857,178.0,143.6100,175.484,1.0,4.0,2.0,R4Grid5sec1Cell25,0.000000,142.339,2.0,2936.2000,38.5909,0.675340,7.45192,main olfactory bulb,None,None,None,None,None,None,NaN,NaN,principal cell,[granule],granule
17960,young adult,Guthrie,"No Diameter, 3D, Angles",[main olfactory bulb],"[principal cell, granule]",2016-01-18,"Dendrites, Soma, No Axon",Transgenic,100,3.0,None,2.0,None,52447,,oil,Neurolucida.dat,Dendrites Complete,in vitro,Neurolucida,[26211445],None,N,Not reported,coronal,160.00,5829.27,mouse,Golgi-Cox,C57BL/6-Tg-(CAM2a-BDNF)A9stl/J,2017-03-30,True,True,59.9956,2.0,0.921029,30.93,0.07,161.8410,1.01961,284.0,162.9300,285.266,2.0,6.0,2.0,R4Grid5Sec1Cell25cell,0.500000,182.972,2.0,5829.2700,62.7332,1.097830,14.78150,main olfactory bulb,None,None,None,None,None,None,NaN,NaN,principal cell,[granule],granule
17961,young adult,Guthrie,"No Diameter, 3D, Angles",[main olfactory bulb],"[principal cell, granule]",2016-01-18,"Dendrites, Soma, No Axon",Transgenic,100,3.0,None,2.0,None,52448,,oil,Neurolucida.dat,Dendrites Complete,in vitro,Neurolucida,[26211445],None,N,Not reported,coronal,160.00,2644.51,mouse,Golgi-Cox,C57BL/6-Tg-(CAM2a-BDNF)A9stl/J,2017-03-30,True,True,68.7565,2.0,0.892786,12.46,0.07,147.6660,1.04314,299.0,152.4200,255.382,3.0,8.0,2.0,TgR4Grid2Sec1Cell1,0.333333,164.089,2.0,2644.5100,56.1614,0.982825,26.83000,main olfactory bulb,None,None,None,None,None,None,NaN,NaN,principal cell,[granule],granule
17962,young adult,Guthrie,"No Diameter, 3D, Angles",[main olfactory bulb],"[principal cell, granule]",2016-01-18,"Dendrites, Soma, No Axon",Transgenic,100,3.0,None,2.0,None,52449,,oil,Ne

# 06 - amygdala

In [61]:
principal_amy = principal_cells[principal_cells['br1'] == 'amygdala']
principal_amy.br2.value_counts(dropna=False)

basolateral amygdala complex    356
Name: br2, dtype: int64

In [62]:
principal_amy.br3.value_counts(dropna=False)

NaN                180
lateral nucleus     88
basal nucleus       88
Name: br3, dtype: int64

In [63]:
principal_amy.cell_type_2_curated.value_counts(dropna=False)

None             177
pyramidal        138
glutamatergic     40
NaN                1
Name: cell_type_2_curated, dtype: int64

## the None and glu neurons in the database should be pyramidal

In [64]:
principal_amy.loc[principal_amy.archive=='Rosenkranz', 'cell_type_2_curated'] = 'pyramidal'
principal_amy.loc[principal_amy.archive=='Rainnie', 'cell_type_2_curated'] = 'pyramidal'
principal_amy = principal_amy[~pd.isnull(principal_amy.cell_type_2_curated)]
principal_amy = principal_amy[principal_amy.cell_type_2_curated != 'None']

/home/yueqi/anaconda/envs/py36/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


## `amygdala` + `pyramidal` could be a small class

In [65]:
principal_amy.cell_type_2_curated.value_counts(dropna=False)

pyramidal    353
Name: cell_type_2_curated, dtype: int64

In [66]:
pd.crosstab(principal_amy.br3, principal_amy.cell_type_2_curated)

cell_type_2_curated,pyramidal
br3,
basal nucleus,87
lateral nucleus,88


# spinal cord neurons -- not enough cells

In [67]:
principal_spinal = principal_cells[principal_cells['br1'] == 'spinal cord']
principal_spinal.br2.value_counts(dropna=False)

lumbar    89
L5-S1     17
L5         3
Name: br2, dtype: int64

In [68]:
principal_spinal.cell_type_2_curated.value_counts(dropna=False)

Motoneuron    61
projection    45
NaN            3
Name: cell_type_2_curated, dtype: int64

In [69]:
principal_spinal = principal_spinal[~pd.isnull(principal_spinal.cell_type_2_curated)]
principal_spinal = principal_spinal[principal_spinal.cell_type_2_curated != 'None']
principal_spinal.cell_type_2_curated.value_counts(dropna=False)

Motoneuron    61
projection    45
Name: cell_type_2_curated, dtype: int64

# brain stem neurons

In [ ]:
principal_bs = principal_cells[principal_cells['br1'] == 'brainstem']
principal_bs.br2.value_counts(dropna=False)

In [ ]:
principal_bs.cell_type_2_curated.value_counts(dropna=False)

In [ ]:
principal_bs = principal_bs[~pd.isnull(principal_bs.cell_type_2_curated)]
principal_bs = principal_bs[principal_bs.cell_type_2_curated != 'None']
principal_bs.cell_type_2_curated.value_counts(dropna=False)

# cerebellum

In [ ]:
principal_cb = principal_cells[principal_cells['br1'] == 'cerebellum']
principal_cb.br2.value_counts(dropna=False)

In [ ]:
principal_cb.cell_type_2_curated.value_counts(dropna=False)

In [ ]:
principal_cb = principal_cb[~pd.isnull(principal_cb.cell_type_2_curated)]
principal_cb = principal_cb[principal_cb.cell_type_2_curated != 'None']
principal_cb.cell_type_2_curated.value_counts(dropna=False)

# Pool all filtered data together

In [ ]:
all_data_list = [principal_cortex, principal_hippo, principal_bg, principal_retina, 
                 principal_ob, principal_amy, principal_spinal, principal_bs, principal_cb]
all_data = pd.concat(all_data_list).sort_values(by='neuron_id').reset_index(drop=True)
all_data.shape

In [ ]:
all_data.head()

In [ ]:
use_data_list = [principal_cortex[principal_cortex.cell_type_2_curated=='pyramidal'], 
                 principal_hippo, 
                 principal_bg[(principal_bg.cell_type_2_curated=='medium spiny') & 
                              principal_bg.br2.isin(['ventral striatum', 'dorsal striatum'])], 
                 principal_retina[principal_retina.cell_type_2_curated=='ganglion'], 
                 principal_ob[principal_ob.cell_type_2_curated=='granule'], 
                 principal_amy[principal_amy.cell_type_2_curated=='pyramidal']]
use_data = pd.concat(use_data_list).sort_values(by='neuron_id').reset_index(drop=True)
use_data.shape

In [ ]:
use_data.loc[pd.isnull(use_data.br2), 'br2'] = 'unknown'
use_data.br2.value_counts(dropna=False)

In [ ]:
pd.crosstab(use_data.br1, use_data.br2)

In [ ]:
pd.crosstab(use_data.cell_type_2_curated, use_data.br1, margins=True)

In [ ]:
pd.crosstab(use_data.cell_type_2_curated, use_data.rough_layer, margins=True)

In [ ]:
pd.crosstab(use_data.cell_type_2_curated, use_data.ds_layer, margins=True)

In [ ]:
pd.crosstab(use_data.cell_type_2_curated, use_data.br2, margins=True)

In [ ]:

all_data.to_pickle('../data/rodent_3d_dendrites_br-ct-filter-3_principal_all.pkl')

In [ ]:
# Almost all neonatal cells come from Hamad

In [ ]:
use_data.age_classification.value_counts(dropna=False)

In [ ]:
use_data[use_data.age_classification=='neonatal'].archive.value_counts(dropna=False)

In [ ]:
# almost all deep/superficial neurons without clear layer label comes from Hamad

In [ ]:
use_data[(pd.isnull(use_data.exact_layer)) & (use_data.ds_layer.isin(['superficial', 'deep']))].archive.value_counts(dropna=False)

# add class label

In [ ]:
use_data['label1'] = 'None'

In [ ]:
use_data.shape

In [ ]:
use_data.loc[use_data.cell_type_2_curated=='medium spiny', 'label1'] = 'striatum medium spiny'
use_data[use_data.cell_type_2_curated=='medium spiny'].shape

In [ ]:
use_data.loc[use_data.cell_type_2_curated=='ganglion', 'label1'] = 'retina ganglion'
use_data[use_data.cell_type_2_curated=='ganglion'].shape

In [ ]:
use_data.loc[(use_data.br1=='hippocampus')&(use_data.cell_type_2_curated=='pyramidal'), 'label1'] = 'hippocampus pyramidal'
use_data[(use_data.br1=='hippocampus')&(use_data.cell_type_2_curated=='pyramidal')].shape

In [ ]:
use_data.loc[(use_data.br1=='hippocampus')&(use_data.cell_type_2_curated=='granule'), 'label1'] = 'hippocampus granule'

use_data[(use_data.br1=='hippocampus')&(use_data.cell_type_2_curated=='granule')].shape

In [ ]:
use_data.loc[(use_data.br1=='neocortex')&(use_data.ds_layer=='deep'), 'label1'] = 'cortex pyramidal deep'
use_data[(use_data.br1=='neocortex')&(use_data.ds_layer=='deep')].shape

In [ ]:
use_data.loc[(use_data.br1=='neocortex')&(use_data.ds_layer=='superficial'), 'label1'] = 'cortex pyramidal superficial'
use_data[(use_data.br1=='neocortex')&(use_data.ds_layer=='superficial')].shape

In [ ]:
use_data.loc[(use_data.br1=='neocortex')&(~use_data.ds_layer.isin(['superficial', 'deep'])), 'label1'] = 'cortex pyramidal no layer'
use_data[(use_data.br1=='neocortex')&(~use_data.ds_layer.isin(['superficial', 'deep']))].shape

In [ ]:
use_data.loc[(use_data.br1=='main olfactory bulb')&(use_data.cell_type_2_curated=='granule'), 'label1'] = 'olfactory bulb granule'
use_data[(use_data.br1=='main olfactory bulb')&(use_data.cell_type_2_curated=='granule')].shape

In [ ]:
use_data.loc[(use_data.br1=='amygdala')&(use_data.cell_type_2_curated=='pyramidal'), 'label1'] = 'amygdala pyramidal'
use_data[(use_data.br1=='amygdala')&(use_data.cell_type_2_curated=='pyramidal')].shape

In [ ]:
use_data.label1.value_counts(dropna=False)

In [ ]:
use_data.to_pickle('../data/rodent_3d_dendrites_br-ct-filter-3_principal_mainclasses.pkl')